In [ ]:
import threading
import time
import os
import random

In [ ]:
lock1 = threading.Lock()
lock2 = threading.Lock()
event = threading.Event()

In [ ]:
class Block:
    def __init__(self, num, status):
        self.number = num
        self.status = status
    def __str__(self):
      return str(self.number) + '(' + str(self.status) + ')'
        

In [ ]:
class BufferCache:
    flag = False
    
    
    def __init__(self, queue, buffers):
        self.free = [Block(-x,2) for x in range(1, (buffers + 1))]
        self.hash = [[] for x in range(queue)]
        
    def insertFree(self, block):
        if block not in self.free:
          self.free.append(block)
        
    def insertHash(self, block):
        b = block.number % (len(self.hash) - 1)
        if block not in self.hash[b]:
          self.hash[b].append(block)
        
    def removeFree(self, block):
        self.free.remove(block)
        
    def deleteFree(self, index):
        block = self.free[index]
        self.free.remove(block)
        if(block.number > 0):
          b = block.number % (len(self.hash) - 1)
          if len(self.hash[b]) != 0:
            self.hash[b].remove(block)
        return block
        
    def search(self, num):
        b = num % (len(self.hash) - 1)
        for i in self.hash[b]:
            if(i.number == num):
                return self.hash[b].index(i)
        return -1
    
    def insertFrontFree(self, block):
        if block not in self.free:
          self.free.insert(0, block)
    
    def brelse(self, block, status):
        if(status):
            block.status = 2
            self.insertFree(block)
        else:
            block.status = 4
            self.insertFrontFree(block)
            
    def diskWrite(self):
        if(self.free[0].status != 4):
            return
        time.sleep(1)
        v = self.free[0]
        self.removeFree(self.free[0])
        time.sleep(2)
        v.status = 2
        self.insertFrontFree(v)
        
    def getblk(self, tempVal, status):
        while(True):
            ind = self.search(tempVal)
            time.sleep(2)

            if(ind != -1):
                b = self.hash[tempVal % (len(self.hash) - 1)][ind]

                # Scenario 5

                if b.status == 1:
                    print('\n Buffer is busy')
                    self.flag = True
                    event.wait()
                    continue

                # Scenario 1

                lock2.acquire()
                b.status = 1
                self.removeFree(b)
                print('\n Locked Buffer:',b.number)
                time.sleep(5)
                self.brelse(b, status)
                print('\n Unlocked Buffer:',b.number)
                time.sleep(5)
                if(self.flag):
                    event.set()
                    event.clear()
                    self.flag = False
                lock2.release()
                break;
            else:
                print('\n Buffer not in cache')

                # Scenario 4

                if len(self.free) == 0:
                    lock2.acquire()
                    tid = threading.Thread(target = self.freeListEmpty)
                    tid.start()
                    tid.join()
                    lock2.release()
                
                index = 0

                # Scenario 3

                while(self.free[index].status == 4):
                    print('\n Asynchronous Write to Disk Cause of Delayed-write in Free List Head Buffer')
                    lock2.acquire()
                    tid = threading.Thread(target = self.diskWrite)
                    tid.start()
                    lock2.release()
                    index += 1

                # Scenario 2

                print('\n Removing 1st Block from Free List')
                oblock = self.deleteFree(index)
                oblock.status = 1
                oblock.number = tempVal
                self.insertHash(oblock)
                print('\n Locked Buffer:',oblock.number)
                time.sleep(5)
                self.brelse(oblock, status)
                print('\n Unlocked Buffer:',oblock.number)
                time.sleep(5)
                if(self.flag):
                    event.set()
                    event.clear()
                    self.flag = False
                break;
        print('Success')
                    
    def freeListEmpty(self):
        print("Free List Empty...")
        
        while(1):
            index = random.randint(0,len(self.hash)-1)
            if len(self.hash[index]) != 0:
                break
        self.lock()
        temp = self.hash[index][0]
        temp.status = 2
        self.insertFree(temp)
        self.unlock()
                    
    def lock(self):
        lock1.acquire()
        
    def unlock(self):
        lock1.release()
        
    def display(self):
        print('\n Buffer Cache')
        print('-------------------------------------------------------')
        k = 0
        for i in self.hash:
            print(' ',end='')
            print(k, end = ' --- ')
            k = k + 1
            for j in i:
                print(j, end = '  ')
            print('\n')
        print('\n Free', end = ' --- ')
        for i in self.free:
            print(i, end = '  ')
        print('\n')
            

In [ ]:
def putBlockNumber(bc):
    value = int(input("\n Enter the Block Number: "))
    st = int(input("\n Enter Read(1) or Write(2):")) # 2 - Means disk write will happen
    status = False
    if st == 1:
      status = True
    elif st == 2:
      pass
    else:
      print('\n Entered the wrong value, start from scratch')
      return
    t1 = threading.Thread(target=bc.getblk, args=(value,status,))
    t1.start()

In [ ]:
def mainLoop(bc):
    
    while(1):
        print("\n 1. Display Current HashQueue")
        print(" 2. Call getblk()")
        print(" 3. Exit")
        ch = int(input("Enter your choice : "))
        if(ch == 1):
            bc.lock()
            bc.display()
            bc.unlock()
        elif(ch == 2):
            putBlockNumber(bc)
        elif(ch == 3):
            os._exit(1)
        else:
            print("\n Invalid input :(")

In [ ]:
if __name__ == "__main__":
    total = int(input('\n Enter the number of Buffers : '))
    h = int(input('\n Enter the number of hashes : '))
    bc = BufferCache(h,total)2
    bc.display()
    mainLoop(bc)